In [2]:
from PIL import Image 
import pytesseract 
import sys
import os 
import pandas as pd
import numpy as np

## Set directory paths

#### One-time setup

In [3]:
#one-time setup
tesseract_path = r'C:\Program Files\Tesseract-OCR\tesseract'

#### Change per AC

In [64]:
#set path to pdfs for this AC (should be in a separate directory)
electoral_rolls_path = r"C:\Users\Naren\Desktop\New folder (3)\New folder\66_3"
#set blank directory for image files converted from pdfs (done automatically)
electoral_roll_images_path = electoral_rolls_path + '_images'
if not os.path.isdir(electoral_roll_images_path) and os.path.isdir(electoral_rolls_path):
    os.mkdir(electoral_roll_images_path)
#set path for spreadsheet of village names picked up by OCR (automatically name by AC)
root_dir = r"C:\Users\Naren\Dropbox\electoral_rolls_matching\up_ocr\include_left\raw"
electoral_roll_names_hin_path = os.path.join(root_dir, electoral_rolls_path.split(os.sep)[-1] + ".xls")

## Convert 1st page of PDFs to JPG

## OCR to get hindi names of village and tehsil and block

In [65]:
os.chdir(electoral_roll_images_path)
pytesseract.pytesseract.tesseract_cmd = tesseract_path

chief_village_names_hin = []
village_names_hin = []
tehsil_names_hin = []
block_names_hin = []
village_id_nos = []

textList = []

i = 1
length = len(os.listdir(electoral_roll_images_path))
while i <  length + 1:
    filename = str(i) + '.jpg'
    if not os.path.isfile(os.path.join(electoral_roll_images_path, filename)):
        length += 1
        i += 1
        continue
    print(i, end = "")
    im = Image.open(filename)
    im_right = im.crop((2715, 1900, 3714, 2827))
    im_left = im.crop((289, 1902, 1716, 3079))
    right_text = str(pytesseract.image_to_string(im_right ,lang='hin')).split('\n')
    left_text = str(pytesseract.image_to_string(im_left ,lang='hin')).split('\n')
    
    j = 0
    while(j < len(right_text)):
        if right_text[j] == '' or right_text[j] == '\x0c':
            right_text.pop(j)
        else:
            j += 1
    j = 0
    while(j < len(left_text)):
        if left_text[j] == '' or left_text[j] == '\x0c':
            left_text.pop(j)
        else:
            j += 1
                
    
    #textList.append(text)
        
    if len(right_text) == 8:
        #no error
        if len(left_text) != 0:
            for name in left_text:
                chief_village_names_hin.append(right_text[0])
                village_names_hin.append(name)
                block_names_hin.append(right_text[2])
                tehsil_names_hin.append(right_text[-4])
                village_id_nos.append(i)
        else:
            village_names_hin.append('')
            chief_village_names_hin.append(right_text[0])
            block_names_hin.append(right_text[2])
            tehsil_names_hin.append(right_text[-4])
            village_id_nos.append(i)
    elif len(right_text) > 0:
        #possible error
        if len(left_text) != 0:
            for name in left_text:
                village_names_hin.append(name)
                chief_village_names_hin.append(right_text[0])
                village_id_nos.append(i)
                block_names_hin.append("block_ocr_failed")
                tehsil_names_hin.append("tehsil_ocr_failed")
        else:
            village_names_hin.append('')
            chief_village_names_hin.append(right_text[0])
            block_names_hin.append("block_ocr_failed")
            tehsil_names_hin.append("tehsil_ocr_failed")
            village_id_nos.append(i)
        print("?", end = "")
    else:
        #mukhya village ocr fail
        if len(left_text) != 0:
            for name in left_text:
                village_names_hin.append(name)
                chief_village_names_hin.append("chief_village_ocr_failed")
                village_id_nos.append(i)
                block_names_hin.append("block_ocr_failed")
                tehsil_names_hin.append("tehsil_ocr_failed")
        else:
            chief_village_names_hin.append("chief_village_ocr_failed")
            village_names_hin.append("")
            village_id_nos.append(i)
            block_names_hin.append("block_ocr_failed")
            tehsil_names_hin.append("tehsil_ocr_failed")
        print("!", end = "")
        
    print(" ", end = "")
    i += 1

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91? 92? 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273? 274? 275? 276

#### Strip leading and trailing whitespace from village and tehsil and block names

In [66]:
for i in range(0, len(village_names_hin)):
    tehsil_names_hin[i] = tehsil_names_hin[i].strip()
    block_names_hin[i] = block_names_hin[i].strip()
    chief_village_names_hin[i] = chief_village_names_hin[i].strip()
    village_names_hin[i] = village_names_hin[i].strip()


## Convert list to dataframe with 5 columns: ps_id, village_name, chief_village_name, tehsil, block and save to excel

In [67]:
#no need to run this if excel file with hindi names already exists
electoral_roll_names_hin = pd.DataFrame({'Name_Cleaned_Hin':village_names_hin})
electoral_roll_names_hin['mukhya_village'] = chief_village_names_hin
electoral_roll_names_hin['tehsil'] = tehsil_names_hin
electoral_roll_names_hin['block'] = block_names_hin
electoral_roll_names_hin['ps_id'] = village_id_nos
#sort records by ps_id
electoral_roll_names_hin = electoral_roll_names_hin.sort_values('ps_id')
electoral_roll_names_hin.to_excel(electoral_roll_names_hin_path, columns = ('ps_id', 'Name_Cleaned_Hin', 'mukhya_village', 'block', 'tehsil'), encoding = 'utf-32',index = False)